# Jupyter notebook sample

In [37]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy
# pip install python-dotenv
# pip install kagglehub

import xml.etree.ElementTree as ET
import pandas as pd
from fontTools.ttLib.tables.S_V_G_ import doc_index_entry_format_0Size
from sodapy import Socrata
from dotenv import load_dotenv
import os
import kagglehub
import ast



In [38]:
# Load environment variables from .env file
load_dotenv()

# Access the secrets
app_token = os.getenv("APP_TOKEN")
username = os.getenv("OPEN_DATA_NYC_USERNAME")
password = os.getenv("OPEN_DATA_NYC_PASSWORD")

In [39]:
def fetch_restaurant_data(app_token, username, password, dataset_id="pitm-atqc", limit=1000):
    """
    Fetch restaurant data from the NYC Open Data API.

    Parameters:
        app_token (str): Your application token for the API.
        username (str): Your username for the API (email).
        password (str): Your password for the API.
        dataset_id (str): The dataset identifier in Socrata.
        limit (int): The maximum number of results to fetch (default is 1000).

    Returns:
        pd.DataFrame: A pandas DataFrame containing the restaurant data.
    """
    # Initialize the Socrata client
    client = Socrata("data.cityofnewyork.us", app_token, username=username, password=password)

    # Fetch data
    results = client.get(dataset_id, limit=limit)

    # Convert results to a pandas DataFrame
    df_restaurants = pd.DataFrame.from_records(results)

    return df_restaurants

# Example usage:
# app_token = "your_app_token"
# username = "your_username"
# password = "your_password"
#df_restaurants = fetch_restaurant_data(app_token, username, password)
# print(df.head())


In [173]:
def parse_selected_sheets(file_path, sheet_names, header_row_mapping):
    """
    Parse selected worksheets from an XML-based Excel workbook.

    Parameters:
        file_path (str): Path to the XML file.
        sheet_names (list): List of worksheet names to parse.
        header_row_mapping (dict): A mapping of sheet names to header row indices.

    Returns:
        list: A list of pandas DataFrames corresponding to the selected sheets.
    """
    def extract_row_data(cells, expected_columns):
        """
        Helper function to extract row data and ensure it matches the number of headers.
        """
        row_data = []
        for i in range(expected_columns):
            try:
                cell = cells[i].find(".//ss:Data", ns)
                row_data.append(cell.text.strip() if cell is not None else None)
            except IndexError:
                row_data.append(None)  # Append None if the column is missing
        return row_data

    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Namespace dictionary for handling XML namespaces
    ns = {'ss': 'urn:schemas-microsoft-com:office:spreadsheet'}

    # Retrieve all worksheets
    worksheets = root.findall(".//ss:Worksheet", ns)

    # List to store DataFrames for selected sheets
    dataframes = []

    # Loop through each worksheet
    for sheet in worksheets:
        sheet_name = sheet.attrib.get(f"{{{ns['ss']}}}Name")  # Get the sheet name
        if sheet_name not in sheet_names:
            continue  # Skip sheets that are not in the specified list

        rows = sheet.findall(".//ss:Row", ns)  # Find all rows in the sheet

        if not rows:
            print(f"Sheet '{sheet_name}' is empty. Skipping...")
            continue

        # Determine the header row index (default to 0 if not specified)
        header_row_index = header_row_mapping.get(sheet_name, 0)
        if header_row_index >= len(rows):
            print(f"Invalid header row index for sheet: {sheet_name}. Skipping...")
            continue

        # Extract headers from the specified row
        header_row = rows[header_row_index]
        headers = []
        for cell in header_row.findall(".//ss:Cell", ns):
            data = cell.find(".//ss:Data", ns)
            headers.append(data.text.strip() if data is not None else None)
        expected_columns = len(headers)

        # Extract data (skip up to the header row)
        data = []
        for row in rows[header_row_index + 1:]:  # Start after the header row
            cells = row.findall(".//ss:Cell", ns)
            row_data = extract_row_data(cells, expected_columns)
            data.append(row_data)

        # Create a DataFrame for the sheet
        df = pd.DataFrame(data, columns=headers)
        dataframes.append(df)  # Add the DataFrame to the list

        print(f"Processed sheet: {sheet_name} with {len(df)} rows")

    return dataframes

def fetch_fliming_locations_data(xml_file_path):
    """
    Fetch the "Full Map List" worksheet as a pandas DataFrame.

    Parameters:
        xml_file_path (str): Path to the XML file.

    Returns:
        pd.DataFrame: A pandas DataFrame containing data from the "Full Map List" worksheet.
    """
    # Sheet names of interest
    selected_sheets = ['Full Map List']

    # Header row mapping for each sheet
    header_row_mapping = {
        'Full Map List': 1
    }

    # Parse the selected sheets
    dfs = parse_selected_sheets(xml_file_path, selected_sheets, header_row_mapping)
    df_filming_locations = dfs[0]  # Extract the DataFrame for the 'Full Map List' sheet

    return df_filming_locations

In [174]:
def fetch_movies_data(kaggle_dataset, filename="25k IMDb movie Dataset.csv"):
    """
    Download the latest version of a Kaggle dataset and return the movies DataFrame.

    Parameters:
        kaggle_dataset (str): The Kaggle dataset identifier (e.g., "utsh0dey/25k-movie-dataset").
        filename (str): The name of the CSV file to load (default is "25k IMDb movie Dataset.csv").

    Returns:
        pd.DataFrame: A pandas DataFrame containing the movies data.
    """
    # Download the latest version of the dataset
    path = kagglehub.dataset_download(kaggle_dataset)
    print("Path to dataset files:", path)

    # Construct the full path to the CSV file
    csv_path = f"{path}/{filename}"

    # Load the dataset into a pandas DataFrame
    df_movies = pd.read_csv(csv_path)

    return df_movies

In [175]:
df_restaurants = fetch_restaurant_data(app_token, username, password)
df_restaurants.head()

,objectid,globalid,seating_interest_sidewalk,restaurant_name,legal_business_name,doing_business_as_dba,bulding_number,street,borough,zip,...,community_board,council_district,census_tract,bin,bbl,nta,roadway_dimensions_length,roadway_dimensions_width,roadway_dimensions_area,landmarkdistrict_terms
0,100,c4b3155b-31a0-4e95-846f-fce09f245437,sidewalk,Pomp and Circumstance Hospitality,Pomp and Circumstance Hospitality LLC,Pomp and Circumstance Hospitality LLC,577,Lorimer Street,Brooklyn,11211,...,1,34,501,3068653,3027560028,East Williamsburg,NaN,NaN,NaN,NaN
1,1000,753495d8-4429-43e5-85a3-dcf6230ef749,both,Charm Kao,193 Schemerhorn INC,Charm Kao,193,Schermerhorn St.,Brooklyn,11201,...,2,33,37,3000493,3001640041,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,24,8,192,NaN
2,10000,{3842B5C5-EF04-41A4-8216-D6EA627DCE5E},openstreets,SAKE BAR HAGI 46,"HAMA NEW YORK, INC.",SAKE BAR HAGI 46,358,W. 46TH STREET,Manhattan,10036,...,4,3,121,1025025,1010360057,Clinton,NaN,NaN,NaN,NaN
3,10001,{C212A0FC-C115-4425-8F95-931B12C5F86A},openstreets,Yum yum too,Boythaicorp,Boythaicorp,662,9ave,Manhattan,10036,...,4,3,127,1025038,1010370001,Clinton,NaN,NaN,NaN,NaN
4,10002,{DA48265D-7730-416F-8E1C-EBC8C8ACE2C2},openstreets,Xochil Pizza Corp,Xochil Pizza Corp,Xochil Pizza Corp,4632,5th Avenue,Brooklyn,11220,...,7,38,80,NaN,NaN,Sunset Park West,NaN,NaN,NaN,NaN


In [176]:
# Example usage
xml_file_path = "./datasets/Interactive_Map_Data.xml"
df_fliming_locations = fetch_fliming_locations_data(xml_file_path)
df_fliming_locations.head()

Processed sheet: Full Map List with 233 rows


,Film,Year,URL Encoded name,Image File Name,Agency Credit,Artist Credit,Director/Filmmaker Name,Director/Filmmaker IMDB Link,Location Display Text,LATITUDE,...,Neighborhood,Scene Type,Media,IMDB LINK,Client or book location indicator,Notes,Book Image,Book Page,Display?,IMAGE OF LOCATION
0,*batteries not included,1987,%2Abatteries%20not%20included,batteriesnotincluded_pf,Courtesy of Photofest,Directed by,Matthew Robbins,http://imdb.com/name/nm0730422/,E. 5th St.<br>East Village<br>Manhattan,40.722445296182798,...,East Village,N/A,Film,http://imdb.com/title/tt0092494/,N/A,None,Y,190,None,Y
1,12 Angry Men,1957,12%20Angry%20Men,12AngryMen_pf,Courtesy of Photofest,Directed by,Sidney Lumet,http://imdb.com/name/nm0001486/,New York County Courthouse<br>40 Foley Square<...,40.713700000000003,...,Lower Manhattan,N/A,Film,http://imdb.com/title/tt0050083/,New York County\nCourthouse on Foley Square.,None,Y,40,None,Y
2,13 Going on 30,2004,13%20Going%20on%2030,13Goingon30_ec,"Courtesy of Everett Collection, Inc.",Directed by,Gary Winick,http://imdb.com/name/nm0935095/,W. 47th St. and Seventh Ave.<br>Times\n ...,40.759220487652101,...,Times Square,N/A,Film,http://www.imdb.com/title/tt0337563/,47th St. and 7th Ave. Times Square\nManhattan,None,Y,239,None,Y
3,15 Minutes,2001,15%20Minutes,None,None,Directed by,John Herzfeld,http://imdb.com/name/nm0381273/,E. 60-66th St. and Madison Ave.<br>Upper East\...,40.766100000000002,...,Upper East Side,Chases,Film,http://www.imdb.com/title/tt0179626/,60-66th and Madison,None,Y,82,None,N
4,25th Hour,2002,25th%20Hour,25thHour1_pf,Courtesy of Photofest,Directed by,Spike Lee,http://www.imdb.com/name/nm0000490/,World Trade Center<br>Lower Manhattan,40.7117926273691,...,Lower Manhattan,N/A,Film,http://www.imdb.com/title/tt0307901/,World Trade Center,None,Y,93,None,Y


In [177]:
# Example usage
kaggle_dataset = "utsh0dey/25k-movie-dataset"
df = fetch_movies_data(kaggle_dataset)

Path to dataset files: C:\Users\huniv\.cache\kagglehub\datasets\utsh0dey\25k-movie-dataset\versions\1


In [123]:
def extract_movie_id(df, path_column='path', new_column='imdb_id'):
    """
    Extract the unique movie ID from the 'path' field and add it as a new column.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing the 'path' column.
        path_column (str): The name of the column containing the path (default: 'path').
        new_column (str): The name of the new column for the extracted movie ID (default: 'movie_id').

    Returns:
        pd.DataFrame: The updated DataFrame with the extracted movie ID column.
    """
    # Check if the path column exists
    if path_column not in df.columns:
        raise ValueError(f"Column '{path_column}' not found in the DataFrame.")

    # Use regex to extract the movie ID from the path
    df[new_column] = df[path_column].str.extract(r'/title/(tt\d+)/')

    return df

In [139]:
def extract_year(df, column_name='year'):
    """
    Extract four-digit year from a given column in the DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the year column.
        column_name (str): The name of the column to process.

    Returns:
        pd.DataFrame: Updated DataFrame with the year column cleaned.
    """
    # Extract the four-digit year using regex
    df[column_name] = df[column_name].str.extract(r'(\b\d{4}\b)', expand=False)

    # Replace any NaN values with "null"
    df[column_name] = df[column_name].fillna("null")

    return df

In [157]:
def prepare_column(df, old_column_name, new_column_name):
    """
    Rename a column and convert its values from strings to Python lists.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the column.
        old_column_name (str): The current name of the column.
        new_column_name (str): The new name for the column.

    Returns:
        pd.DataFrame: The updated DataFrame with the renamed and properly formatted column.

    Raises:
        ValueError: If the old_column_name is not found in the DataFrame.
    """
    if old_column_name not in df.columns:
        raise ValueError(f"Column '{old_column_name}' not found in the DataFrame.")

    # Rename the column
    df = df.rename(columns={old_column_name: new_column_name})

    # Convert column values from string to list
    df[new_column_name] = df[new_column_name].apply(ast.literal_eval)  # Safely convert string to list

    return df


def create_lookup_table(df, column_name, id_column_name, value_column_name):
    """
    Create a lookup table with unique values and their IDs from a column in the DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the column.
        column_name (str): The name of the column to extract unique values from.
        id_column_name (str): The name of the ID column in the lookup table.
        value_column_name (str): The name of the value column in the lookup table.

    Returns:
        pd.DataFrame: A lookup table with unique values and their IDs.
    """
    # Extract unique values and assign IDs
    unique_values = set(value for values_list in df[column_name] for value in values_list)
    lookup_table = pd.DataFrame({value_column_name: sorted(unique_values)})
    lookup_table[id_column_name] = lookup_table.index + 1  # Assign unique IDs starting from 1

    return lookup_table


def create_link_table(df, lookup_table, column_name, id_column_name, movie_id_column, value_column_name):
    """
    Create a link table connecting movies to values (e.g., genres, actors) by their IDs.

    Parameters:
        df (pd.DataFrame): The movies DataFrame.
        lookup_table (pd.DataFrame): The lookup table with unique values and their IDs.
        column_name (str): The name of the column in the movies DataFrame to link.
        id_column_name (str): The name of the ID column in the link table.
        movie_id_column (str): The name of the unique movie identifier column in the movies DataFrame.
        value_column_name (str): The name of the value column in the lookup table.

    Returns:
        pd.DataFrame: A link table connecting movies (movie_id) to values (e.g., genres, actors) by their IDs.
    """
    # Explode the column into separate rows
    df_expanded = df.explode(column_name)

    # Map values to their IDs using the lookup table
    link_table = (df_expanded[[movie_id_column, column_name]]
                  .merge(lookup_table, left_on=column_name, right_on=value_column_name)
                  .rename(columns={id_column_name: id_column_name})
                  )

    # Drop unnecessary columns and return the link table
    return link_table[[movie_id_column, id_column_name]]

def clean_and_reorder_movies(df):
    """
    Clean and reorder the movies DataFrame by renaming, dropping, and reordering columns.

    Parameters:
        df (pd.DataFrame): The input movies DataFrame.

    Returns:
        pd.DataFrame: The cleaned and reordered movies DataFrame.
    """
    # Rename columns, drop unuseful columns, make lowercase, and reorder
    df = (
        df.rename(columns={
            'movie title': 'title',
            'User Rating': 'nb_users_ratings',
            'Rating': 'rating'
        })
        .drop(columns=['Run Time', 'genres', 'Plot Kyeword', 'actors', 'path'])
        .pipe(lambda x: x.set_axis(x.columns.str.lower(), axis=1))
        .loc[:, ['imdb_id', 'title', 'year', 'director', 'writer', 'overview', 'rating', 'nb_users_ratings']]
    )
    return df

def process_movie_data(df_movies):
    """
    Process the movie dataset through the entire data pipeline:
    1. Prepare and clean the genres column.
    2. Extract the 4-digit year.
    3. Create the Genres lookup table.
    4. Create the Movies_Genres link table.
    5. Prepare and clean the actors column.
    6. Create the Actors lookup table.
    7. Create the Movies_Actors link table.
    8. Clean and reorder the movies DataFrame.

    Parameters:
        df_movies (pd.DataFrame): The raw movies DataFrame.

    Returns:
        tuple: A tuple containing:
            - df_movies (pd.DataFrame): Cleaned and reordered movies DataFrame.
            - df_genres (pd.DataFrame): Genres lookup table.
            - df_movies_genres (pd.DataFrame): Movies_Genres link table.
            - df_actors (pd.DataFrame): Actors lookup table.
            - df_movies_actors (pd.DataFrame): Movies_Actors link table.
    """
    # Step 1: Prepare the genres column
    df_movies = prepare_column(df_movies, old_column_name='Generes', new_column_name='genres')

    # Step 2: Extract 4-digit year
    df_movies = extract_year(df_movies, column_name='year')

    # Step 3: Create the Genres lookup table
    df_genres = create_lookup_table(df_movies, column_name='genres', id_column_name='genre_id', value_column_name='genre')

    # Step 4: Create the Movies_Genres link table
    df_movies_genres = create_link_table(
        df_movies,
        df_genres,
        column_name='genres',
        id_column_name='genre_id',
        movie_id_column='imdb_id',
        value_column_name='genre'
    )

    # Step 5: Prepare the actors column
    df_movies = prepare_column(df_movies, old_column_name='Top 5 Casts', new_column_name='actors')

    # Step 6: Create the Actors lookup table
    df_actors = create_lookup_table(
        df_movies, column_name='actors', id_column_name='actor_id', value_column_name='actor_name'
    )

    # Step 7: Create the Movies_Actors link table
    df_movies_actors = create_link_table(
        df_movies,
        df_actors,
        column_name='actors',
        id_column_name='actor_id',
        movie_id_column='imdb_id',
        value_column_name='actor_name'
    )

    # Step 8: Clean and reorder the movies DataFrame
    df_movies = clean_and_reorder_movies(df_movies)

    return df_movies, df_genres, df_movies_genres, df_actors, df_movies_actors



In [158]:
df_movies = df
# Assuming the required functions are already implemented:
# - prepare_column
# - extract_year
# - create_lookup_table
# - create_link_table
# - clean_and_reorder_movies

# Process the movie data
df_movies_cleaned, df_genres, df_movies_genres, df_actors, df_movies_actors = process_movie_data(df_movies)

# Outputs
print("Movies DataFrame:")
print(df_movies_cleaned)

print("\nGenres Table:")
print(df_genres)

print("\nMovies_Genres Table:")
print(df_movies_genres)

print("\nActors Table:")
print(df_actors)

print("\nMovies_Actors Table:")
print(df_movies_actors)


Movies DataFrame:
          imdb_id                    title  year              director  \
0       tt1745960        Top Gun: Maverick  2022       Joseph Kosinski   
1       tt8041270  Jurassic World Dominion  2022       Colin Trevorrow   
2       tt0092099                  Top Gun  1986            Tony Scott   
3      tt10298810                Lightyear  2022         Angus MacLane   
4       tt9783600               Spiderhead  2022       Joseph Kosinski   
...           ...                      ...   ...                   ...   
24397   tt0101700             Delicatessen  1991             Marc Caro   
24398  tt13991504                Bitch Ass  2022           Bill Posley   
24399   tt0051438                 Bullwhip  1958          Harmon Jones   
24400   tt0099615             The Freshman  1990        Andrew Bergman   
24401   tt0048140           Guys and Dolls  1955  Joseph L. Mankiewicz   

                   writer                                           overview  \
0            